# 

In [1]:
import csv
import time
import pandas as pd
import numpy as np
from scipy import interp
from math import isnan
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc, f1_score
from sklearn.externals import joblib
from xgboost import XGBClassifier
from lightgbm import LGBMModel
from lightgbm import LGBMClassifier
import lightgbm
import graphviz

In [2]:
cd D:\aectdata\original

D:\aectdata\original


In [3]:
def label_transformation(train):
    train['label'][train.score == -1]=1
    
def fillna(train,test):
    train = train.fillna(value=-1)
    test = test.fillna(value=-1)

In [4]:
def fs_lgb(train,test, threshold):
    y=train['label']
    train_nolabel=train.drop('label',axis=1)
    X0 = train_nolabel.iloc[:, :].as_matrix()
    test0=test.iloc[:,:].as_matrix()
    clf = LGBMClassifier(n_estimators=100, n_jobs=-1, silent=True)
    clf.fit(X0, y)
    sfm = SelectFromModel(clf, prefit=True, threshold=threshold)
    X = sfm.transform(X0)
    fe_ind=sfm.get_support(indices=True)
    fe_name=[]
    for i in range(len(fe_ind)):
        feature='f%s'%(fe_ind[i]+1)
        fe_name.append(feature)
    test_reduced = test[fe_name]
    return X, test_reduced

def lgblgb_testA():
    print('reading training data')
    train = pd.read_csv('atec_anti_fraud_train.csv', index_col='id')
    print('reading test data')
    test = pd.read_csv('atec_anti_fraud_test_a.csv', index_col='id')
    
    print('**********************model1*****************************')
    
    print('data clean')
    train = train.sort_values('date')
    train['label'][train.label == -1]=1
    
    train = train.fillna(value=-1)
    test = test.fillna(value=-1)
    
    week1_train=train.loc[train['date']<20170912]
    rest_train=train.loc[(train['date']>=20170912) & (train['date']<20171010)]
    week1_test=test.loc[test['date']<20180109]
    rest_test=test.loc[test['date']>=20180109]
    
    testdate=test['date']
    testdate=testdate.apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    test['f298'] = testdate.dt.dayofweek
    
    week1_train = week1_train.drop('date',axis=1)
    rest_train = rest_train.drop('date',axis=1)
    week1_test = week1_test.drop('date',axis=1)
    rest_test = rest_test.drop('date',axis=1)
    
    train = train.drop('date',axis=1)
    test = test.drop('date',axis=1)
    
    print('feature selection')
    X_wk1, test_wk1 = fs_lgb(week1_train,week1_test, '0.1*mean')
    y_wk1 = week1_train['label']
    X_wk2345, test_wk2345 = fs_lgb(rest_train, rest_test, '0.1*mean')
    y_rest = rest_train['label']
    
    clf_wk1 = LGBMClassifier(n_estimators=200, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=11)
    clf_wk2345 = LGBMClassifier(n_estimators=1000, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=18)
    
    print('fitting models')
    model_wk1 = clf_wk1.fit(X_wk1, y_wk1)
    model_rest = clf_wk2345.fit(X_wk2345, y_rest)
    
    print('predicting results')
    prob_wk1 = model_wk1.predict_proba(test_wk1)
    prob_wk2345 = model_rest.predict_proba(test_wk2345)
    
    result_wk1=pd.DataFrame(prob_wk1[:,1],index=week1_test.index,columns=['score_wk1234'])
    result_rest = pd.DataFrame(prob_wk2345[:,1],index=rest_test.index, columns=['score_wk1234'])

    #print('write to csv')
    prob_result_1 = pd.concat([result_wk1,result_rest])
    
    ### weekday:
    print('***************************model2**************************')
    
    X, test_wkd = fs_lgb(train,test,'0.1*mean')
    y = train['label']
    clf =  LGBMClassifier(n_estimators=900, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=18)
    model_wkd = clf.fit(X,y)
    prob_wkd = model_wkd.predict_proba(test_wkd)
    result_wkd = pd.DataFrame(prob_wkd[:,1],index=test_wkd.index,columns=['score_wkd'])
    
    prob_result_2 = result_wkd
    
    print('fusion')
    
    pred_result = pd.merge(prob_result_1,prob_result_2,left_index=True,right_index=True)
    pred_result['score']=pred_result['score_wk1234']*0.6 + pred_result['score_wk1234']*0.4
    pred_result = pred_result.drop('score_wk1234',axis=1)
    pred_result = pred_result.drop('score_wkd',axis=1)
    
    return pred_result

def fs_gbdt(train,test, threshold):
    y=train['label']
    train_nolabel=train.drop('label',axis=1)
    X0 = train_nolabel.iloc[:, :].as_matrix()
    test0=test.iloc[:,:].as_matrix()
    clf = GradientBoostingClassifier(n_estimators=50, random_state=100)
    clf.fit(X0, y)
    sfm = SelectFromModel(clf, prefit=True, threshold=threshold)
    X = sfm.transform(X0)
    fe_ind=sfm.get_support(indices=True)
    fe_name=[]
    for i in range(len(fe_ind)):
        feature='f%s'%(fe_ind[i]+1)
        fe_name.append(feature)
    test_reduced = test[fe_name]
    return X, test_reduced

    
def gbdtlgb_AB():
    testA = pd.read_csv('atec_anti_fraud_test_a.csv', index_col='id')
    test = pd.read_csv('atec_anti_fraud_test_b.csv', index_col='id')
    A_result = lgblgb_testA()
    A_result['label']=0
    A_result['label'][A_result.score>0.5]=1
    train=pd.merge(testA,A_result,left_index=True,right_index=True)
    train=train.drop(['score'],axis=1)
    
    print('data clean')
    train = A.sort_values('date')
    
    train = train.fillna(value=-1)
    test = test.fillna(value=-1)
    
    train = train.drop('date',axis=1)
    test = test.drop('date',axis=1)
    
    print('feature selection')
    X, testB = fs_gbdt(train,test, '0.1*mean')
    y = train['label']
    
    print('fitting model')
    clf =  LGBMClassifier(n_estimators=900, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=18)
    model = clf.fit(X,y)
    prob_AB = model.predict_proba(testB)
    result_AB = pd.DataFrame(prob_AB[:,1],index=test.index,columns=['score_AB'])
    return result_AB



def lgblgb_B_wk1vswk234():
    print('reading training data')
    train = pd.read_csv('atec_anti_fraud_train.csv', index_col='id')
    print('reading test data')
    test = pd.read_csv('atec_anti_fraud_test_b.csv', index_col='id')
    
    print('data clean')
    train = train.sort_values('date')
    train['label'][train.label == -1]=1
    
    train = train.fillna(value=-1)
    test = test.fillna(value=-1)
    
    week1_train=train.loc[train['date']<20170912]
    rest_train=train.loc[(train['date']>=20170912) & (train['date']<20171009)]
    week1_test=test.loc[test['date']<20180210]
    rest_test=test.loc[test['date']>=20180210]
    
    week1_train = week1_train.drop('date',axis=1)
    rest_train = rest_train.drop('date',axis=1)
    week1_test = week1_test.drop('date',axis=1)
    rest_test = rest_test.drop('date',axis=1)
    
    s = 10
    np.random.seed(s)
    
    print('feature selection')
    X_wk1, test_wk1 = fs_lgb(week1_train,week1_test, '0.1*mean')
    y_wk1 = week1_train['label']
    X_wk2345, test_wk2345 = fs_lgb(rest_train, rest_test, '0.1*mean')
    y_rest = rest_train['label']
    
    clf_wk1 = LGBMClassifier(n_estimators=200, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=11)
    clf_wk2345 = LGBMClassifier(n_estimators=1000, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=11)
    
    print('fitting models')
    model_wk1 = clf_wk1.fit(X_wk1, y_wk1)
    model_rest = clf_wk2345.fit(X_wk2345, y_rest)
    
    print('predicting results')
    prob_wk1 = model_wk1.predict_proba(test_wk1)
    prob_wk2345 = model_rest.predict_proba(test_wk2345)
    
    result_wk1=pd.DataFrame(prob_wk1[:,1],index=week1_test.index,columns=['score_B'])
    result_rest = pd.DataFrame(prob_wk2345[:,1],index=rest_test.index, columns=['score_B'])

    print('write to csv')
    prob_result_B = pd.concat([result_wk1,result_rest])
    
    prob_result_AB=gbdtlgb_AB()
    
    pred_result = pd.merge(prob_result_B,prob_result_AB,left_index=True,right_index=True)
    pred_result['score']=pred_result['score_B']*0.5 + pred_result['score_AB']*0.5
    pred_result = pred_result.drop('score_AB',axis=1)
    pred_result = pred_result.drop('score_B',axis=1)
    
    
    #csvfile = open('result', 'w', encoding='utf8', newline='')
    prob_result.to_csv('we_are_the_champions.csv')
    
#if __name__ == '__main__':

 #   lgblgb_B_wk1vswk234()

In [5]:
def lgblgb_testA():
    print('reading training data')
    train = pd.read_csv('atec_anti_fraud_train.csv', index_col='id')
    print('reading test data')
    test = pd.read_csv('atec_anti_fraud_test_a.csv', index_col='id')
    
    print('**********************model1*****************************')
    
    print('data clean')
    train = train.sort_values('date')
    train['label'][train.label == -1]=1
    
    train = train.fillna(value=-1)
    test = test.fillna(value=-1)
    
    week1_train=train.loc[train['date']<20170912]
    rest_train=train.loc[(train['date']>=20170912) & (train['date']<20171010)]
    week1_test=test.loc[test['date']<20180109]
    rest_test=test.loc[test['date']>=20180109]
    
    testdate=test['date']
    testdate=testdate.apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    test['f298'] = testdate.dt.dayofweek
    
    week1_train = week1_train.drop('date',axis=1)
    rest_train = rest_train.drop('date',axis=1)
    week1_test = week1_test.drop('date',axis=1)
    rest_test = rest_test.drop('date',axis=1)
    
    train = train.drop('date',axis=1)
    test = test.drop('date',axis=1)
    
    print('feature selection')
    X_wk1, test_wk1 = fs_lgb(week1_train,week1_test, '0.1*mean')
    y_wk1 = week1_train['label']
    X_wk2345, test_wk2345 = fs_lgb(rest_train, rest_test, '0.1*mean')
    y_rest = rest_train['label']
    
    clf_wk1 = LGBMClassifier(n_estimators=200, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=11)
    clf_wk2345 = LGBMClassifier(n_estimators=1000, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=18)
    
    print('fitting models')
    model_wk1 = clf_wk1.fit(X_wk1, y_wk1)
    model_rest = clf_wk2345.fit(X_wk2345, y_rest)
    
    print('predicting results')
    prob_wk1 = model_wk1.predict_proba(test_wk1)
    prob_wk2345 = model_rest.predict_proba(test_wk2345)
    
    result_wk1=pd.DataFrame(prob_wk1[:,1],index=week1_test.index,columns=['score_wk1234'])
    result_rest = pd.DataFrame(prob_wk2345[:,1],index=rest_test.index, columns=['score_wk1234'])

    #print('write to csv')
    prob_result_1 = pd.concat([result_wk1,result_rest])
    
    ### weekday:
    print('***************************model2**************************')
    
    X, test_wkd = fs_lgb(train,test,'0.1*mean')
    y = train['label']
    clf =  LGBMClassifier(n_estimators=900, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=18)
    model_wkd = clf.fit(X,y)
    prob_wkd = model_wkd.predict_proba(test_wkd)
    result_wkd = pd.DataFrame(prob_wkd[:,1],index=test_wkd.index,columns=['score_wkd'])
    
    prob_result_2 = result_wkd
    
    print('fusion')
    
    pred_result = pd.merge(prob_result_1,prob_result_2,left_index=True,right_index=True)
    pred_result['score']=pred_result['score_wk1234']*0.6 + pred_result['score_wk1234']*0.4
    pred_result = pred_result.drop('score_wk1234',axis=1)
    pred_result = pred_result.drop('score_wkd',axis=1)
    
    return pred_result

def fs_gbdt(train,test, threshold):
    y=train['label']
    train_nolabel=train.drop('label',axis=1)
    X0 = train_nolabel.iloc[:, :].as_matrix()
    test0=test.iloc[:,:].as_matrix()
    clf = GradientBoostingClassifier(n_estimators=50, random_state=100)
    clf.fit(X0, y)
    sfm = SelectFromModel(clf, prefit=True, threshold=threshold)
    X = sfm.transform(X0)
    fe_ind=sfm.get_support(indices=True)
    fe_name=[]
    for i in range(len(fe_ind)):
        feature='f%s'%(fe_ind[i]+1)
        fe_name.append(feature)
    test_reduced = test[fe_name]
    return X, test_reduced

    
def gbdtlgb_AB():
    testA = pd.read_csv('atec_anti_fraud_test_a.csv', index_col='id')
    test = pd.read_csv('atec_anti_fraud_test_b.csv', index_col='id')
    A_result = lgblgb_testA()
    A_result['label']=0
    A_result['label'][A_result.score>0.5]=1
    train=pd.merge(testA,A_result,left_index=True,right_index=True)
    train=train.drop(['score'],axis=1)
    
    print('data clean')
    train = A.sort_values('date')
    
    train = train.fillna(value=-1)
    test = test.fillna(value=-1)
    
    train = train.drop('date',axis=1)
    test = test.drop('date',axis=1)
    
    print('feature selection')
    X, testB = fs_gbdt(train,test, '0.1*mean')
    y = train['label']
    
    print('fitting model')
    clf =  LGBMClassifier(n_estimators=900, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=18)
    model = clf.fit(X,y)
    prob_AB = model.predict_proba(testB)
    result_AB = pd.DataFrame(prob_AB[:,1],index=test.index,columns=['score_AB'])
    return result_AB
    

In [6]:
rst = lgblgb_testA()

reading training data
reading test data
**********************model1*****************************
data clean


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


feature selection


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


fitting models
predicting results
***************************model2**************************


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


fusion


In [6]:
train = pd.DataFrame(pd.read_csv('atec_anti_fraud_train.csv', index_col='id', nrows=2000))
test = pd.DataFrame(pd.read_csv('atec_anti_fraud_test_b.csv',nrows=1000, index_col='id'))

In [7]:
train['label'][train.label == -1]=1
train = train.fillna(value=-1)
test = test.fillna(value=-1)
train = train.drop(['date'],axis=1)
test = test.drop(['date'],axis=1)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X0 = train.iloc[:, :].as_matrix()
y=train.label
clf = LGBMClassifier(n_estimators=100, n_jobs=-1, silent=True)
clf.fit(X0, y)
sfm = SelectFromModel(clf, prefit=True, threshold='0.1*mean')
X = sfm.transform(X0)

In [ ]:
fe=sfm.get_support(indices=True)
print(len(fe))
print(fe)

In [ ]:
X, test_reduced = fs_lgb(train,test,'0.1*mean')
clf_wk1 = LGBMClassifier(n_estimators=200, learning_rate=0.06, reg_lambda=20, silent=True,num_leaves=11)
model_wk1 = clf_wk1.fit(X, y)
prob_wk1 = model_wk1.predict_proba(test_reduced)

In [ ]:
test['result']=pd.DataFrame(prob_wk1[:,1],dtype='float64'),ignore_index=True

In [ ]:
prob=pd.DataFrame(prob_wk1[:,1],index=test.index,columns=['score'])

#prob.rename(columns={0:'score'})


In [ ]:
prob.head()

In [ ]:
featurename=['f1','f5']
test[featurename]

In [8]:
X, test_reduced = fs_lgb(train,test, '0.1*mean')

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [ ]:
print(np.shape(X))
print(np.shape(test_reduced))

In [10]:
test_reduced

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f218,f219,f226,f238,f242,f254,f260,f265,f279,f291
id,,,,,,,,,,,,,,,,,,,,,
661ae9a03c6c74feab2555c9987e140ae3f5421bd8e7a078d35bcf54f419619f,0,0,1,1,100810.0,0,4,2,0,1,...,11.0,2.0,4.0,7.0,2.0,2.0,2.0,2.0,302.0,192.0
ada98d4358e72a27cc5e92f691a87a4fd62a7cda0387e213caa193052bc26f23,0,0,1,1,100810.0,1,6,1,0,2,...,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,5.0
e33675a962e5bf44d05a2b01903a4beb88a0c6385c05c6be576d7d829f96c505,0,0,1,1,100811.0,1,2,1,0,1,...,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,90.0,130.0
4c5da176de7172dbd1ad0aa7edf9866548ec720d6c73186e4382fa28a0bed92a,1,0,1,0,100811.0,0,5,1,2,1,...,2.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,301.0,256.0
fa39e3491900d49b862d30b5dbcd0b1c30bb4ff0d963966cf55bcee796fab7eb,0,0,1,1,100811.0,1,7,1,1,2,...,10.0,2.0,1.0,9.0,2.0,1.0,10.0,14.0,222.0,108.0
ae0f1ecb27e098bbf672529a50237dff27abd16b34956980aaeb77362a918b6f,1,1,0,0,100109.0,1,7,2,0,1,...,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0
7a9e5e415412a92928f91608c39750e3a33eb123eb0ef8cfd3b5a7504e4d38af,2,0,1,1,-1.0,4,1,1,1,1,...,301.0,1.0,2.0,301.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
51fb55256e959c36989151e8136a8a68002bb75887eb75d186915a4e479adf04,0,1,0,0,100808.0,0,2,1,1,1,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,78.0,83.0
aa7664463d11ae5573ae6d89195373fbe033c3547101424023b6ce96d3e6d4cc,0,0,1,1,100811.0,1,2,2,0,0,...,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,78.0,62.0
